# SLB Well Casing and TIE Segmentation - Data Exploration

This notebook explores the ultrasonic well imaging dataset for the SLB data challenge. 

**Challenge Goal:** Detect well casing and Third Interface Echo (TIE) in ultrasonic images for well integrity assessment.

**Dataset Overview:**
- 11 wells total (6 training, 5 test)
- Ultrasonic patches of size 160×160 or 160×272
- Multi-class segmentation masks (background, casing, TIE)
- Evaluation metric: Intersection over Union (IoU)

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set up visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Define data paths
data_dir = Path('data')
x_train_dir = data_dir / 'x_train_images'
x_test_dir = data_dir / 'x_test_images'
y_train_file = data_dir / 'y_train_labels' / 'Y_train_T9NrBYo.csv'

print("Libraries imported successfully!")
print(f"Data directory: {data_dir}")
print(f"Training images dir: {x_train_dir}")
print(f"Labels file: {y_train_file}")

## 2. Load Training Data

In [ ]:
# Load training labels from CSV
print("Loading training labels...")
y_train = pd.read_csv(y_train_file, index_col=0)
print(f"Loaded {len(y_train)} training patches")
print(f"\nDataset shape: {y_train.shape}")
print(f"Sample patch names: {list(y_train.index[:5])}")
print(f"\nFirst few values of first patch:")
print(y_train.iloc[0, :20])

In [ ]:
# Function to load and process a single patch
def load_patch(patch_name, x_dir=x_train_dir):
    """Load a single patch image from .npy file"""
    patch_path = x_dir / f"{patch_name}.npy"
    if patch_path.exists():
        return np.load(patch_path)
    else:
        print(f"Warning: Patch {patch_name} not found")
        return None

# Function to process labels for a patch
def load_label(patch_name, y_dataframe):
    """Load and reshape labels for a single patch, removing padding"""
    if patch_name in y_dataframe.index:
        label_values = np.array([v for v in y_dataframe.loc[patch_name] if v != -1])
        # Try to determine shape
        if len(label_values) == 160 * 160:
            return label_values.reshape(160, 160)
        elif len(label_values) == 160 * 272:
            return label_values.reshape(160, 272)
        else:
            return label_values.reshape(160, -1)
    return None

# Test loading a patch
test_patch_name = y_train.index[0]
print(f"Testing with patch: {test_patch_name}")
test_image = load_patch(test_patch_name)
test_label = load_label(test_patch_name, y_train)

print(f"Image shape: {test_image.shape if test_image is not None else 'None'}")
print(f"Label shape: {test_label.shape if test_label is not None else 'None'}")
print(f"Image value range: [{test_image.min():.2f}, {test_image.max():.2f}]" if test_image is not None else "")

## 3. Explore Data Structure and Dimensions

In [ ]:
# Extract well and section information from patch names
wells_info = defaultdict(lambda: {'sections': set(), 'patches': []})

for patch_name in y_train.index:
    parts = patch_name.split('_')
    well_num = parts[1]  # Extract well number from "well_X"
    section_num = parts[3]  # Extract section number from "section_Y"
    
    wells_info[well_num]['sections'].add(int(section_num))
    wells_info[well_num]['patches'].append(patch_name)

# Display summary
print("=" * 70)
print("TRAINING DATA SUMMARY")
print("=" * 70)

summary_data = []
for well in sorted(wells_info.keys(), key=lambda x: int(x)):
    info = wells_info[well]
    num_patches = len(info['patches'])
    num_sections = len(info['sections'])
    summary_data.append({
        'Well': well,
        'Sections': num_sections,
        'Patches': num_patches,
        'Approx. Size': f"~{num_patches//num_sections}x{num_sections if num_sections == 18 else 36}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print(f"\nTotal Training Patches: {len(y_train)}")
print(f"Total Training Wells: {len(wells_info)}")

In [ ]:
# Check patch dimensions by sampling patches
patch_dimensions = defaultdict(int)

print("\nAnalyzing patch dimensions...")
for i, patch_name in enumerate(y_train.index[:100]):  # Sample first 100 patches
    label_values = np.array([v for v in y_train.loc[patch_name] if v != -1])
    num_pixels = len(label_values)
    
    if num_pixels == 160 * 160:
        patch_dimensions['160x160'] += 1
    elif num_pixels == 160 * 272:
        patch_dimensions['160x272'] += 1
    else:
        patch_dimensions[f'Other ({num_pixels})'] += 1

print("\nPatch Dimensions Distribution (sample):")
for dim, count in sorted(patch_dimensions.items()):
    print(f"  {dim}: {count} patches")

## 4. Visualize Sample Patches and Masks

In [ ]:
# Create a color map for segmentation classes
class_colors = {
    0: 'black',      # Background
    1: 'red',        # Casing
    2: 'blue'        # TIE (Third Interface Echo)
}

class_names = {
    0: 'Background',
    1: 'Casing',
    2: 'TIE'
}

# Visualize samples from different wells
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Sample Ultrasonic Images and Their Segmentation Masks', fontsize=16, fontweight='bold')

well_samples = ['1', '2', '3', '4', '5', '6']
sample_count = 0

for well in well_samples:
    # Get patches from this well
    well_patches = [p for p in y_train.index if f'well_{well}_' in p]
    
    if well_patches:
        # Sample middle patch from well
        sample_idx = len(well_patches) // 2
        patch_name = well_patches[sample_idx]
        
        image = load_patch(patch_name)
        label = load_label(patch_name, y_train)
        
        if image is not None and label is not None:
            # Plot image
            row = sample_count // 2
            col = (sample_count % 2) * 2
            
            if row < 3:
                # Raw image
                ax1 = axes[row, col]
                im1 = ax1.imshow(image, cmap='gray')
                ax1.set_title(f'Well {well} - Raw Image\n{patch_name}', fontsize=10)
                ax1.set_xlabel('Width (pixels)')
                ax1.set_ylabel('Height (pixels)')
                plt.colorbar(im1, ax=ax1, label='Intensity')
                
                # Segmentation mask
                ax2 = axes[row, col+1]
                im2 = ax2.imshow(label, cmap='RdYlBu_r', vmin=0, vmax=2)
                ax2.set_title(f'Segmentation Mask\nClasses: {len(np.unique(label))}', fontsize=10)
                ax2.set_xlabel('Width (pixels)')
                ax2.set_ylabel('Height (pixels)')
                cbar = plt.colorbar(im2, ax=ax2, label='Class')
                cbar.set_ticks([0, 1, 2])
                cbar.set_ticklabels(['Background', 'Casing', 'TIE'])
                
                sample_count += 1

plt.tight_layout()
plt.show()

print("\nVisualization complete!")

## 5. Analyze Class Distribution

In [ ]:
# Analyze class distribution across all training patches
print("Analyzing class distribution across training data...")
print("This may take a moment...\n")

class_counts = {0: 0, 1: 0, 2: 0}  # Background, Casing, TIE
patch_class_presence = {0: 0, 1: 0, 2: 0}  # Count of patches containing each class

for i, patch_name in enumerate(y_train.index):
    if (i + 1) % 500 == 0:
        print(f"  Processed {i+1}/{len(y_train)} patches...", end='\r')
    
    label_values = np.array([v for v in y_train.loc[patch_name] if v != -1])
    
    for class_id in [0, 1, 2]:
        count = np.sum(label_values == class_id)
        class_counts[class_id] += count
        if count > 0:
            patch_class_presence[class_id] += 1

print(f"  Processed {len(y_train)}/{len(y_train)} patches... Done!     \n")

# Calculate statistics
total_pixels = sum(class_counts.values())
total_patches = len(y_train)

print("\n" + "=" * 70)
print("CLASS DISTRIBUTION STATISTICS")
print("=" * 70)

for class_id in [0, 1, 2]:
    pixel_pct = (class_counts[class_id] / total_pixels) * 100
    patch_pct = (patch_class_presence[class_id] / total_patches) * 100
    
    print(f"\nClass {class_id} ({class_names[class_id]}):")
    print(f"  Total pixels: {class_counts[class_id]:,} ({pixel_pct:.2f}%)")
    print(f"  Patches containing: {patch_class_presence[class_id]:,} ({patch_pct:.2f}%)")
    
print(f"\nTotal pixels analyzed: {total_pixels:,}")
print(f"Total patches: {total_patches:,}")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Pixel distribution
pixel_percentages = [(class_counts[i] / total_pixels) * 100 for i in [0, 1, 2]]
colors_plot = ['lightgray', 'red', 'blue']
labels_plot = [f'{class_names[i]}\n({pixel_percentages[i]:.2f}%)' for i in range(3)]

axes[0].pie(pixel_percentages, labels=labels_plot, colors=colors_plot, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Pixel Distribution by Class', fontsize=12, fontweight='bold')

# Plot 2: Patch presence distribution
patch_percentages = [(patch_class_presence[i] / total_patches) * 100 for i in [0, 1, 2]]
axes[1].bar(range(3), [class_counts[i] / 1e6 for i in range(3)], color=colors_plot)
axes[1].set_xlabel('Class')
axes[1].set_ylabel('Pixel Count (Millions)')
axes[1].set_title('Total Pixel Count by Class', fontsize=12, fontweight='bold')
axes[1].set_xticks(range(3))
axes[1].set_xticklabels([class_names[i] for i in range(3)])
axes[1].grid(axis='y', alpha=0.3)

for i, (count, pct) in enumerate(zip([class_counts[j] / 1e6 for j in range(3)], pixel_percentages)):
    axes[1].text(i, count + 0.1, f'{pct:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Examine Data Statistics per Well

In [ ]:
# Detailed statistics per well
print("\nCalculating per-well statistics...\n")

well_statistics = []

for well in sorted(wells_info.keys(), key=lambda x: int(x)):
    well_patches = wells_info[well]['patches']
    
    # Count pixels per class in this well
    well_class_counts = {0: 0, 1: 0, 2: 0}
    well_patch_class_presence = {0: 0, 1: 0, 2: 0}
    patch_dims = defaultdict(int)
    
    for patch_name in well_patches:
        label_values = np.array([v for v in y_train.loc[patch_name] if v != -1])
        num_pixels = len(label_values)
        
        # Count dimension types
        if num_pixels == 160 * 160:
            patch_dims['160x160'] += 1
        elif num_pixels == 160 * 272:
            patch_dims['160x272'] += 1
        
        for class_id in [0, 1, 2]:
            count = np.sum(label_values == class_id)
            well_class_counts[class_id] += count
            if count > 0:
                well_patch_class_presence[class_id] += 1
    
    # Calculate percentages
    total_well_pixels = sum(well_class_counts.values())
    
    well_stat = {
        'Well': well,
        'Patches': len(well_patches),
        'Sections': len(wells_info[well]['sections']),
        'Casing %': (well_class_counts[1] / total_well_pixels * 100) if total_well_pixels > 0 else 0,
        'TIE %': (well_class_counts[2] / total_well_pixels * 100) if total_well_pixels > 0 else 0,
        'Dominant Size': max(patch_dims.items(), key=lambda x: x[1])[0] if patch_dims else 'N/A'
    }
    
    well_statistics.append(well_stat)

# Create DataFrame
well_stats_df = pd.DataFrame(well_statistics)
print("=" * 80)
print("STATISTICS PER TRAINING WELL")
print("=" * 80)
print(well_stats_df.to_string(index=False))

print("\n\nKey Observations:")
print(f"  • Wells with most patches: Well {well_stats_df.loc[well_stats_df['Patches'].idxmax(), 'Well']} ({well_stats_df['Patches'].max()} patches)")
print(f"  • Wells with least patches: Well {well_stats_df.loc[well_stats_df['Patches'].idxmin(), 'Well']} ({well_stats_df['Patches'].min()} patches)")
print(f"  • Average casing pixels per well: {well_stats_df['Casing %'].mean():.2f}%")
print(f"  • Average TIE pixels per well: {well_stats_df['TIE %'].mean():.2f}%")

## 7. Visualize Multiple Wells and Sections

In [ ]:
# Visualize patches from different sections of a single well
fig = plt.figure(figsize=(16, 10))
fig.suptitle('Well 1: Patches from Different Sections (Angular Views)', fontsize=16, fontweight='bold')

well_1_patches = [p for p in y_train.index if 'well_1_' in p]
sections_to_show = [0, 4, 8, 12, 16]  # Different sections/azimuths

displayed = 0
for section in sections_to_show:
    # Get a patch from this section
    section_patches = [p for p in well_1_patches if f'section_{section}_' in p]
    
    if section_patches:
        # Use the first patch from this section
        patch_name = section_patches[0]
        image = load_patch(patch_name)
        label = load_label(patch_name, y_train)
        
        if image is not None and label is not None:
            # Plot image
            ax1 = plt.subplot(2, 5, displayed + 1)
            im1 = ax1.imshow(image, cmap='gray')
            ax1.set_title(f'Section {section} - Raw Image', fontsize=10)
            ax1.set_ylabel('Depth (pixels)')
            if displayed == 0:
                plt.colorbar(im1, ax=ax1, label='Intensity')
            
            # Plot mask
            ax2 = plt.subplot(2, 5, displayed + 6)
            im2 = ax2.imshow(label, cmap='RdYlBu_r', vmin=0, vmax=2)
            ax2.set_title(f'Section {section} - Mask', fontsize=10)
            ax2.set_ylabel('Depth (pixels)')
            if displayed == 0:
                cbar = plt.colorbar(im2, ax=ax2, label='Class')
                cbar.set_ticks([0, 1, 2])
                cbar.set_ticklabels(['Bg', 'Cas', 'TIE'], fontsize=8)
            
            displayed += 1

plt.tight_layout()
plt.show()

print(f"Displayed {displayed} different sections from Well 1")

## 8. Data Exploration Summary and Next Steps

In [ ]:
print("\n" + "=" * 80)
print("EXPLORATION SUMMARY")
print("=" * 80)

print("\n📊 DATASET OVERVIEW:")
print(f"  • Total training patches: {len(y_train):,}")
print(f"  • Training wells: 6 (Wells 1-6)")
print(f"  • Test wells: 5 (Wells 7-11)")
print(f"  • Total wells: 11")

print("\n🖼️ IMAGE CHARACTERISTICS:")
print(f"  • Patch types: 160×160 and 160×272 pixels")
print(f"  • Ultrasonic intensity: Float values")
print(f"  • Multiple sections per well (18 or 36 angular views)")

print("\n🎯 SEGMENTATION CLASSES:")
print(f"  • Class 0 (Background): {class_counts[0]:,} pixels ({(class_counts[0]/total_pixels*100):.2f}%)")
print(f"  • Class 1 (Casing): {class_counts[1]:,} pixels ({(class_counts[1]/total_pixels*100):.2f}%)")
print(f"  • Class 2 (TIE): {class_counts[2]:,} pixels ({(class_counts[2]/total_pixels*100):.2f}%)")

print("\n⚠️ KEY CHALLENGES:")
print(f"  • Class imbalance: Casing/TIE are {(class_counts[0]/max(class_counts[1], class_counts[2])):.1f}x less frequent than background")
print(f"  • Variable patch sizes: Mix of 160×160 and 160×272 images")
print(f"  • Different well characteristics: Class distribution varies significantly between wells")

print("\n✅ NEXT STEPS FOR MODELING:")
print("  1. Data preprocessing: Normalize intensities, handle variable sizes")
print("  2. Address class imbalance: Use weighted loss, data augmentation, or resampling")
print("  3. Model architecture: Consider U-Net or similar segmentation networks")
print("  4. Evaluation metric: Focus on IoU score (mean IoU across all test patches)")
print("  5. Post-processing: Resize predictions back to original dimensions if needed")

print("\n" + "=" * 80)